### Question 3 : Time Series Analysis for Sales Forecasting

Can we develop a reliable time series model to forecast future sales volumes and identify 
seasonal patterns in the Brazilian e-commerce market?

#### CSV Structure

olist_customers_dataset.csv

    customer_id – unique per order

    customer_unique_id – consistent across all orders from a customer

    customer_zip_code_prefix – first digits of Brazilian zip codes

    customer_city and customer_state


olist_geolocation_dataset.csv

    geolocation_zip_code_prefix – same format as customer zip

    geolocation_lat, geolocation_lng – latitude & longitude

    geolocation_city, geolocation_state

Data

In [5]:
import pandas as pd


customers_df = pd.read_csv("../Dataset/raw/olist_customers_dataset.csv")
geolocation_df = pd.read_csv("../Dataset/raw/olist_geolocation_dataset.csv")


Clean data

In [ ]:
# Check for and remove duplicates if necessary
print("Duplicate customer_id:", customers_df.duplicated(subset='customer_id').sum())
print("Duplicate customer_unique_id:", customers_df.duplicated(subset='customer_unique_id').sum())

# check for negative zipcode values
assert (customers_df['customer_zip_code_prefix'] >= 0).all(), "Invalid ZIP codes found!"

# missing values
print("Missing values in customers:\n", customers_df.isnull().sum())


Duplicate customer_id: 0
Duplicate customer_unique_id: 3345
Missing values in customers:
 customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64


No missing values found. 
Duplicate unique customers ids means that a customer may have placed more than one order. With the question on focus, duplicate values should not be dropped.